<a href="https://colab.research.google.com/github/YoonTae-Hwang/trading_repository/blob/master/3_Sector_Momentum_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The Strategy : Sector Momentum Rotation Strategy

1. Sector Momentum Rotation Strategy Introduction 
 - 우선, 주식시장에서 Sector이란 무엇인가?
    - The Global Industry Classification Standard 따른 기업의 산업은 총 11개의 Sector로 구분 할 수 있으며, 세부적으로 각 섹터안에서 산업구분은 24개, 68개 157개로 구분지을 수도 있다. [Link](https://analyzingalpha.com/stock-sectors)
  - 역사적으로 각각의 Sector들은 서로다른 비즈니스 사이클을 가지고 있다. 즉, 개별 종목들은 다양한 이유로 위험이 존재하는것에 반해 Sector들은 위험이 분산효과로 인해 상당히 사라지고 이로인해 Sector 고유의 모멘텀 효과만 얻을 수 있다. 
  - 정리하자면, Sector Momentum Rotation 다음과 같은 가정을 한다.
    1. 시장에는 독립적으로 움직이는 여러 Sector가 존재한다.
    2. 시장에는 언제나 기회가 있다. 예를들어 아무리 2008 금융위기에서 주식으로인한 손실은 컸지만 안전자산으로 대표되는 금의 수익률은 높았다.
    3. 비지니스 사이클이 존재해서, 상대적으로 저평가된 섹터가 강세장(Bullish)으로 전환되기 직전에 들어가서 섹터가 약세장(Bearish)으로 전환되기 전에 빠져나올 수 있다.[Q. 기업에 모멘텀이 존재하는가?](https://www.jstor.org/stable/798005?seq=1)
    4. 가장 실적이 좋은 섹터에 집중된 포트폴리오를 유지하면 돈을 벌 수 있다. ![사진](./strategy_img/sector-Momentum-Rotation.jpg)


2. Sector Momentum Rotation Strategy Method
 - 전략은 다음과 같은 방법으로 수행된다.
    - 1. momentum calculation : 90-day slope calculation
    - 2. lookback window : 1999-2020
    - 3. trading strategy : Long position
    - 4. ranking percentage & number of assets : optimzie
    - 5. benchmark : SPY
  - [What Momentum investing?](https://www.youtube.com/watch?time_continue=1904&v=F2TTfZzigYk&feature=emb_logo)

In [65]:
!pip install backtrader
!pip install yfinance
#python library
import pandas as pd
import numpy as np
import backtrader as bt
from scipy.stats import linregress
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime
import yfinance as yf

In [68]:
# Date range
start = '2010-01-01'
end = '2020-03-25'

ETF_TICKERS = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY',"SPY"]

prices = yf.download(ETF_TICKERS, start = start, end = end)
prices = prices.dropna()

def momentum_func(self, price_array):
    r = np.log(price_array)
    slope, _, rvalue, _, _ = linregress(np.arange(len(r)), r)
    annualized = (1 + slope) ** 252
    return (annualized * (rvalue ** 2))


class Momentum(bt.ind.OperationN):
    lines = ('trend',)
    params = dict(period=90)
    func = momentum_func


class Strategy(bt.Strategy):
    params = dict(
        momentum=Momentum,
        momentum_period=180,
        num_positions=2,
        when=bt.timer.SESSION_START,
        timer=True,
        monthdays=[1],
        monthcarry=True,
        printlog=True
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.i = 0
        self.securities = self.datas[1:]
        self.inds = {}

        self.add_timer(
            when=self.p.when,
            monthdays=self.p.monthdays,
            monthcarry=self.p.monthcarry
        )

        for security in self.securities:
            self.inds[security] = self.p.momentum(security,
                                                  period=self.p.momentum_period)

    def notify_timer(self, timer, when, *args, **kwargs):
        if self._getminperstatus() < 0:
            self.rebalance()

    def rebalance(self):
        rankings = list(self.securities)
        rankings.sort(key=lambda s: self.inds[s][0], reverse=True)
        pos_size = 1 / self.p.num_positions

        # Sell stocks no longer meeting ranking filter.
        for i, d in enumerate(rankings):
            if self.getposition(d).size:
                if i > self.p.num_positions:
                    self.close(d)

        # Buy and rebalance stocks with remaining cash
        for i, d in enumerate(rankings[:self.p.num_positions]):
            self.order_target_percent(d, target=pos_size)

    def next(self):
        self.notify_timer(self, self.p.timer, self.p.when)

    def stop(self):
        self.log('| %2d | %2d |  %.2f |' %
                 (self.p.momentum_period,
                  self.p.num_positions,
                  self.broker.getvalue()),
                 doprint=True)


if __name__ == '__main__':
    assets_prices = []

    for i in ETF_TICKERS[1:len(ETF_TICKERS) - 1]:
        prices_ = prices.drop(columns='Adj Close').loc[:, (slice(None), i)].dropna()
        prices_.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
        assets_prices.append(bt.feeds.PandasData(dataname=prices_, plot=True))

    # Creating Benchmark bt.feeds
    benchdata = prices.drop(columns='Adj Close').loc[:, (slice(None), 'SPY')].dropna()
    benchdata.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
    benchdata = bt.feeds.PandasData(dataname=benchdata, plot=True)

    cerebro = bt.Cerebro(optreturn=False)

    for data in benchdata:
        cerebro.adddata(benchdata)

    for data in assets_prices:
        data.plotinfo.plotmaster = benchdata
        cerebro.adddata(data)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Add Strategy
    stop = len(ETF_TICKERS) + 1
    cerebro.optstrategy(Strategy,
                        momentum_period=range(50, 300, 50),
                        num_positions=range(1, len(ETF_TICKERS) + 1))

    # Run the strategy. Results will be output from stop.
    cerebro.run(stdstats=False, tradehistory=False)

[*********************100%***********************]  10 of 10 completed
Starting Portfolio Value: 10000.00
2020-03-24, | 50 |  1 |  13022.67 |
2020-03-24, | 50 |  2 |  15278.37 |
2020-03-24, | 50 |  3 |  17539.95 |
2020-03-24, | 50 |  4 |  17400.63 |
2020-03-24, | 50 |  5 |  17822.25 |
2020-03-24, | 50 |  6 |  21763.75 |
2020-03-24, | 50 |  7 |  22213.68 |
2020-03-24, | 50 |  8 |  20330.85 |
2020-03-24, | 50 |  9 |  18989.28 |
2020-03-24, | 50 | 10 |  17872.85 |
2020-03-24, | 100 |  1 |  18527.36 |
2020-03-24, | 100 |  2 |  24319.71 |
2020-03-24, | 100 |  3 |  23869.19 |
2020-03-24, | 100 |  4 |  25206.93 |
2020-03-24, | 100 |  5 |  23916.91 |
2020-03-24, | 100 |  6 |  24039.45 |
2020-03-24, | 100 |  7 |  23291.96 |
2020-03-24, | 100 |  8 |  21224.98 |
2020-03-24, | 100 |  9 |  19657.98 |
2020-03-24, | 100 | 10 |  18452.02 |
2020-03-24, | 150 |  1 |  22408.38 |
2020-03-24, | 150 |  2 |  26325.40 |
2020-03-24, | 150 |  3 |  24881.90 |
2020-03-24, | 150 |  4 |  24926.91 |
2020-03-24, | 15